In [23]:
import pandas as pd
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import text
from sklearn.feature_extraction import _stop_words as stop_words

from sklearn.metrics import accuracy_score,recall_score,precision_score, confusion_matrix

import json
import os


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
master_df = pd.read_csv("link to training data")

In [6]:
len(master_df)

1799

In [11]:
#set feature and targets
X = master_df[['Post Text', 'Length of Title']]
y = master_df['Subreddit']
for index, row in X.iterrows():
  row['Post Text'] = row['Post Text'][: row['Length of Title']]

In [13]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify= y)

In [15]:
#show us the shape of our data
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1349, 2)
(1349,)
(450, 2)
(450,)


In [16]:
additional_politics_english_stop = ['www', 'things', 'does', 'x200b', 'amp', 'want', 'watch',
                           'just', 'like', 'https', 'com', 'republican', 'republicans',
                           'libertarians', 'democrats', 'democrat', 'people', 'libertarian',
                           'says', 'say', 'did', 'this', 'conservative', 'conservatives' ]

additional_english_stop = ['www', 'things', 'does', 'x200b', 'amp',
                           'just', 'like', 'https', 'com', 'watch', 'want',
                           'says', 'say', 'did', 'this']

new_stop_list = stop_words.ENGLISH_STOP_WORDS.union(additional_english_stop)
new_politics_english_stop_list = stop_words.ENGLISH_STOP_WORDS.union(additional_politics_english_stop)
print(len(stop_words.ENGLISH_STOP_WORDS))
print(len(additional_english_stop))
print(len(new_politics_english_stop_list))
print(len(new_stop_list))



318
15
341
332


In [17]:
pipe_cvec_lr = Pipeline([
    ('cvec', CountVectorizer()),
    ('lr', LogisticRegression())
])

pipe_params_cvec_lr = {
    'cvec__max_features': [None,500,1000],
    'cvec__min_df': [2,3],
    'cvec__max_df': [.3,.4,],
    'cvec__ngram_range': [(1,2),(1,3)],
    'cvec__stop_words': [None,'english',new_stop_list],
    'lr__penalty': ['l2']
}

gs = GridSearchCV(pipe_cvec_lr, param_grid=pipe_params_cvec_lr, cv=5,n_jobs = -1,verbose = 1)

gs.fit(X_train['Post Text'],y_train)


Fitting 5 folds for each of 72 candidates, totalling 360 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
120 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
67 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/usr/local/lib/python3.10/dist-p

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('cvec', CountVectorizer()),
                                       ('lr', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'cvec__max_df': [0.3, 0.4],
                         'cvec__max_features': [None, 500, 1000],
                         'cvec__min_df': [2, 3],
                         'cvec__ngram_range': [(1, 2), (1, 3)],
                         'cvec__stop_words': [None, 'english',
                                              frozenset({'a', 'about', 'above',
                                                         'across', 'after',
                                                         'afterwards', 'again',
                                                         'against', 'all',
                                                         'almost', 'alone',
                                                         'along', 'already',
                                                         'also', 'although',
                                                         'always', 'am',
                                                         'among', 'amongst',
                                                         'amoungst', 'amount',
                                                         'amp', 'an', 'and',
                                                         'another', 'any',
                                                         'anyhow', 'anyone',
                                                         'anything', 'anyway', ...})],
                         'lr__penalty': ['l2']},
             verbose=1)

In [18]:
cvlr_bestscore = gs.best_score_
cvlr_params = gs.best_params_
cvlr_train = gs.score(X_train["Post Text"],y_train)
cvlr_test= gs.score(X_test["Post Text"],y_test)
cvlr = ('CountVec with LogReg', cvlr_bestscore, cvlr_params, cvlr_train, cvlr_test)


In [19]:
print(f'Best CV Score: {gs.best_score_}')
print(f'Best Parameters: {gs.best_params_}')
print(f'Train Accuracy Score: {gs.score(X_train["Post Text"],y_train)}')
print(f'Test Accuracy Score: {gs.score(X_test["Post Text"],y_test)}')

Best CV Score: 0.7828142640782046
Best Parameters: {'cvec__max_df': 0.4, 'cvec__max_features': None, 'cvec__min_df': 3, 'cvec__ngram_range': (1, 2), 'cvec__stop_words': 'english', 'lr__penalty': 'l2'}
Train Accuracy Score: 0.9599703484062269
Test Accuracy Score: 0.7977777777777778


In [27]:
def make_prediction_json(filename, dir_path):
  title_file = open(dir_path + "/" + filename, 'r')
  titles = title_file.readlines()
  title_length = [len(title) for title in titles]

  dict = {'Post Text': titles, 'Length of Title': title_length}
  df = pd.DataFrame(dict)

  predictions = gs.best_estimator_.predict(df['Post Text'])
  prediction_list = []
  for i in range(len(predictions)):
    pred = "democrat"
    if predictions[i] == 1:
      pred = "republican"
    prediction_list.append({"title": titles[i], "prediction": pred})
  # print(prediction_politics_list[1])
  with open(dir_path + "/" + filename + "_prediction.json", "w") as jsonf:
    json.dump(prediction_list, jsonf)

In [21]:
# with open('/content/drive/MyDrive/JumboHack 2024/prediction_politics.json', "w") as jsonf:
#     json.dump(prediction_politics_list, jsonf)

In [28]:
dir_path = '/content/drive/MyDrive/JumboHack 2024/all_titles_x'
for filename in os.listdir(dir_path):
    if filename.startswith("all_titles"):
        make_prediction_json(filename, dir_path)